In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('white')

In [2]:
import warnings
warnings.filterwarnings("ignore")
import scanpy.api as sc
from scanpy.neighbors import Neighbors
import scanorama

# Evaluating Alignment Score For Batch Correction

This notebook evaluates an "alignment score" to quantify the effects of batch correction. This notebook uses this metric to measure the performance of the Scanorama algorithm on two batches of Olivier's CAOV3 data with one batch receiving an addition of random gaussian noise.

# Load Data

In [3]:
SENS03_B1 = sc.read_10x_mtx("../data/Harismendy_data/170206/SENS03", var_names="gene_symbols")
SENS03_B2 = sc.read_10x_mtx("../data/Harismendy_data/170315/SENS03", var_names="gene_symbols")

## Downsample For Performance

In [4]:
def downsample_anndata(ad, n):
    rand_obs = np.random.choice(list(ad.obs.index), size=n, replace=False)
    return ad[rand_obs,:]

In [5]:
type(SENS03_B1)

anndata.base.AnnData

In [6]:
SENS03_B1 = downsample_anndata(SENS03_B1, 500)
SENS03_B2 = downsample_anndata(SENS03_B2, 500)

In [7]:
SENS03_B1.shape

(500, 33694)

In [8]:
SENS03_B1.shape

(500, 33694)

# Implement Alignment Score

In [9]:
INPUT_ANNS = [SENS03_B1, SENS03_B1]

In [10]:
min_cells = min(list(map(lambda x: x.n_obs, INPUT_ANNS)))
DS_ANNS = list(map(lambda x: downsample_anndata(x, min_cells), INPUT_ANNS))

## Merge Experiments

In [11]:
COMB_ANN = DS_ANNS[0].concatenate(DS_ANNS[1:], index_unique="_")
COMB_ANN.shape

(1000, 33694)

## Find Closest 1% Neighbors For All Cells

In [12]:
n_batch = len(INPUT_ANNS)

In [13]:
n_neighbors = max(int(0.01*min_cells), 10)
n_neighbors

10

In [14]:
sc.pp.neighbors(COMB_ANN, n_neighbors=22, knn=True)

         Falling back to preprocessing with `sc.pp.pca` and default params.


## Calculate Alignment Score

In [15]:
rows, cols = COMB_ANN.uns['neighbors']['distances'].nonzero()

In [26]:
# this includes a distance matrix over all cells
COMB_ANN.uns['neighbors']

{'params': {'n_neighbors': 22, 'method': 'umap'},
 'distances': <1000x1000 sparse matrix of type '<class 'numpy.float64'>'
 	with 20386 stored elements in Compressed Sparse Row format>,
 'connectivities': <1000x1000 sparse matrix of type '<class 'numpy.float64'>'
 	with 31456 stored elements in Compressed Sparse Row format>}

In [29]:
rows[100:120]

array([4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], dtype=int32)

In [30]:
cols[100:120]

array([868, 900,  98, 135, 172, 272, 284, 292, 359, 369, 371, 417, 506,
       537, 615, 633, 674, 696, 770, 793], dtype=int32)

In [ ]:
def get_batch(i, adata):
    """
    Returns the batch number of a cell at index i in adata
    """
    return int(adata.obs.index[i].split('_')[1])

In [ ]:
neighbor_counts = {}  #{cell_id: #same_batch}   

for x, y in zip(rows, cols):
    try:
        neighbor_counts[x]
    except KeyError:
        neighbor_counts[x] = 0
    cell_batch = get_batch(x, COMB_ANN)
    neigh_batch = get_batch(y, COMB_ANN)   ## what does this do? how are neighbors used?
    if cell_batch == neigh_batch:   ## WHY?
        neighbor_counts[x] += 1

In [ ]:
(np.mean(list(neighbor_counts.values()))/n_neighbors)*n_batch

# Summary Implementation Function

This cell defines a single function to run the above analysis given a list of AnnData

In [ ]:
def downsample_anndata(ad, n):
    rand_obs = np.random.choice(list(ad.obs.index), size=n, replace=False)
    return ad[rand_obs,:]

def get_batch(i, adata):
    """
    Returns the batch number of a cell at index i in adata
    """
    return int(adata.obs.index[i].split('_')[1])

def score_batch_corr(anns, n_neighbors):
    min_cells = min(list(map(lambda x: x.n_obs, anns)))
    ds_anns = list(map(lambda x: downsample_anndata(x, min_cells), anns))
    comb_ann = ds_anns[0].concatenate(ds_anns[1:], index_unique="_")
    sc.tl.pca(comb_ann)
    sc.pp.neighbors(comb_ann, n_neighbors=n_neighbors, knn=True)
    rows, cols = comb_ann.uns['neighbors']['distances'].nonzero()
    neighbor_counts = {}  #{cell_id: #same_batch}
    for x, y in zip(rows, cols):
        try:
            neighbor_counts[x]
        except KeyError:
            neighbor_counts[x] = 0
        cell_batch = get_batch(x, comb_ann)
        neigh_batch = get_batch(y, comb_ann)
        if cell_batch != neigh_batch:
            neighbor_counts[x] += 1
    sc.pl.pca(comb_ann, color='batch', show=True, save=False)
    return (np.mean(list(neighbor_counts.values()))/n_neighbors)*len(anns)

In [ ]:
N_NEIGHBORS = int(0.01*SENS03_B1.shape[0])
N_NEIGHBORS = 10

In [ ]:
score_batch_corr([SENS03_B1, SENS03_B2], N_NEIGHBORS)

# Add Random Gaussian Noise To Olivier's Data

In [ ]:
rows, cols = SENS03_B2.X.nonzero()
count = 0
for x, y in zip(rows, cols):

    SENS03_B2.X[x, y] += np.random.normal(loc=50, scale=100)
    count += 1
    if count%100000 == 0:
        print(float(count)/len(rows)*100)

In [ ]:
score_batch_corr([SENS03_B1, SENS03_B2], N_NEIGHBORS)

# Correct Data With Scanorama

In [ ]:
SENS03_B1, SENS03_B2 = scanorama.correct_scanpy([SENS03_B1, SENS03_B2])
score_batch_corr([SENS03_B1, SENS03_B2], N_NEIGHBORS)